In [2]:
#spacy
import spacy
from spacy.pipeline import EntityRuler
from spacy.lang.en import English
from spacy.tokens import Doc

#gensim
import gensim
from gensim import corpora

#Visualization
from spacy import displacy
import pyLDAvis.gensim_models
from wordcloud import WordCloud
import plotly.express as px
import matplotlib.pyplot as plt

#Data loading/ Data manipulation
import pandas as pd
import numpy as np
import jsonlines

#nltk
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download(['stopwords','wordnet'])

#warning
import warnings 
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rajchhabria/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rajchhabria/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
df = pd.read_csv('Resume.csv')
df.head()

,Category,link,id,Resume,Raw_html
0,hr,https://www.livecareer.com/resume-search/r/hr-...,203361909970992332506290823189098544432,HR EMPLOYEE SERVICE REPRESENTATIVE ...,"<div class=""fontsize fontface vmargins hmargin..."
1,hr,https://www.livecareer.com/resume-search/r/bri...,176452925245576845188344162105053478091,"BRIDGES OF AMERICA, HR SPECIALIST II ...","<div class=""fontsize fontface vmargins hmargin..."
2,hr,https://www.livecareer.com/resume-search/r/reg...,224895221270146291614712443228903825828,REGIONAL HR MANAGER Summary ...,"<div class=""fontsize fontface vmargins hmargin..."
3,hr,https://www.livecareer.com/resume-search/r/hr-...,120168133438024183006274626332357739684,HR GENERALIST Professional Prof...,"<div class=""fontsize fontface vmargins hmargin..."
4,hr,https://www.livecareer.com/resume-search/r/hr-...,37082950160311880080159288707042371082,HR COORDINATOR Summary To o...,"<div class=""fontsize fontface vmargins hmargin..."


In [4]:
df.shape

(277, 5)

In [5]:
df = df[['id','Category','Resume','Raw_html']]
df.head()

,id,Category,Resume,Raw_html
0,203361909970992332506290823189098544432,hr,HR EMPLOYEE SERVICE REPRESENTATIVE ...,"<div class=""fontsize fontface vmargins hmargin..."
1,176452925245576845188344162105053478091,hr,"BRIDGES OF AMERICA, HR SPECIALIST II ...","<div class=""fontsize fontface vmargins hmargin..."
2,224895221270146291614712443228903825828,hr,REGIONAL HR MANAGER Summary ...,"<div class=""fontsize fontface vmargins hmargin..."
3,120168133438024183006274626332357739684,hr,HR GENERALIST Professional Prof...,"<div class=""fontsize fontface vmargins hmargin..."
4,37082950160311880080159288707042371082,hr,HR COORDINATOR Summary To o...,"<div class=""fontsize fontface vmargins hmargin..."


In [10]:
nlp = spacy.load("en_core_web_lg")
skill_pattern_path = "jz_skill_patterns.jsonl"

In [7]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.8 MB 6.2 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [8]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 400.7 MB 7.4 kB/s eta 0:00:01    |█▏                              | 14.0 MB 4.0 MB/s eta 0:01:38     |█▉                              | 23.0 MB 4.1 MB/s eta 0:01:32     |██▍                             | 29.7 MB 4.1 MB/s eta 0:01:32     |██▌                             | 30.6 MB 4.1 MB/s eta 0:01:31     |███▋                            | 45.3 MB 5.8 MB/s eta 0:01:02     |████                            | 51.3 MB 4.2 MB/s eta 0:01:23     |███████▏                        | 90.1 MB 3.6 MB/s eta 0:01:27     |██████████▎                     | 128.6 MB 4.8 MB/s eta 0:00:58     |██████████▍                     | 129.4 MB 4.8 MB/s eta 0:00:58     |███████████▉                    | 148.0 MB 2.6 MB/s eta 0:01:38     |███████████▉                    | 148.8 MB 2.6 MB/s eta 0:01:38     |████████████▏                   | 152.8 MB 3.8 MB/s eta 0:01:05     |████████████▍                   | 155.4 MB 3.1 MB/s eta 0:01:19     |███████████████▊                | 19

In [23]:
ruler = nlp.add_pipe("entity_ruler")
ruler.from_disk(skill_pattern_path)
nlp.pipe_names


ValueError: [E007] 'entity_ruler' already exists in pipeline. Existing names: ['tok2vec', 'tagger', 'parser', 'senter', 'attribute_ruler', 'lemmatizer', 'ner', 'entity_ruler']

In [15]:
def get_skills(text):
    doc = nlp(text)
    myset = []
    subset = []
    for ent in doc.ents:
        if ent.label_ == "SKILL":
            subset.append(ent.text)
    myset.append(subset)
    return subset


def unique_skills(x):
    return list(set(x))

In [18]:
clean = []
for i in range(df.shape[0]):
    review = re.sub(
        '(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"',
        " ",
        df["Resume"].iloc[i],
    )
    review = review.lower()
    review = review.split()
    lm = WordNetLemmatizer()
    review = [
        lm.lemmatize(word)
        for word in review
        if not word in set(stopwords.words("english"))
    ]
    review = " ".join(review)
    clean.append(review)

In [19]:
df["Clean_Resume"] = clean
df["skills"] = df["Clean_Resume"].str.lower().apply(get_skills)
df["skills"] = df["skills"].apply(unique_skills)
df.head()

,id,Category,Resume,Raw_html,Clean_Resume,skills
0,203361909970992332506290823189098544432,hr,HR EMPLOYEE SERVICE REPRESENTATIVE ...,"<div class=""fontsize fontface vmargins hmargin...",hr employee service representative summary exc...,[]
1,176452925245576845188344162105053478091,hr,"BRIDGES OF AMERICA, HR SPECIALIST II ...","<div class=""fontsize fontface vmargins hmargin...",bridge america hr specialist ii hr benefit spe...,[]
2,224895221270146291614712443228903825828,hr,REGIONAL HR MANAGER Summary ...,"<div class=""fontsize fontface vmargins hmargin...",regional hr manager summary holistic hr profes...,[]
3,120168133438024183006274626332357739684,hr,HR GENERALIST Professional Prof...,"<div class=""fontsize fontface vmargins hmargin...",hr generalist professional profile responsible...,[]
4,37082950160311880080159288707042371082,hr,HR COORDINATOR Summary To o...,"<div class=""fontsize fontface vmargins hmargin...",hr coordinator summary obtain human resource g...,[]


In [22]:
df['skills']

0      []
1      []
2      []
3      []
4      []
       ..
272    []
273    []
274    []
275    []
276    []
Name: skills, Length: 277, dtype: object